In [28]:
import pymysql
import pandas as pd
import requests
import os

In [30]:
def get_connection():
   connection = pymysql.connect(host='deadhead-db.cplvgriavgfs.us-east-1.rds.amazonaws.com',
                             user= db_username,
                             password= db_password,
                             database='deadhead',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)
   return connection

In [31]:
def get_concert():
  conn = get_connection()
  with conn:
     with conn.cursor() as cursor:
        query = "SELECT * FROM concert ORDER BY RAND() LIMIT 1"
        cursor.execute(query)
        concert = cursor.fetchone()
        print(concert)
        return concert

In [32]:
def get_song(concert):
    concert_id = concert['concert_id']
    conn = get_connection()
    with conn:
      with conn.cursor() as cursor:
          query = "SELECT * FROM song WHERE `concert_id` = {} ORDER BY RAND() LIMIT 1".format(concert_id)
          cursor.execute(query)
          song = cursor.fetchone()
          if song is None:
             print("No song in this concert")
          print(song)
          return song

In [33]:
concert = get_concert()

{'concert_id': 370, 'concert_title': 'Grateful Dead Live at Frost Amphitheatre - Stanford University on 1983-08-21', 'year': 1983, 'coverage': 'Palo Alto, CA', 'concert_archive_id': 'gd1983-08-21.fob.sonyECM220t.kirschner.miller.95687.flac24', 'date': '1983-08-21T00:00:00Z'}


In [34]:
song = get_song(concert)

{'song_title': None, 'format': 'Flac', 'song_archive_id': '/gd83-08-21s2t09.flac', 'song_url': 'https://ia804601.us.archive.org/0/items/gd1983-08-21.fob.sonyECM220t.kirschner.miller.95687.flac24/gd83-08-21s2t09.flac', 'concert_id': 370, 'corrupted': 'No'}


## Tests

### Check if link is bad

In [35]:
def get_all_songs():
  conn = get_connection()
  with conn:
      with conn.cursor() as cursor:
        query = "SELECT * FROM song"
        cursor.execute(query)
        songs = cursor.fetchall()
        return songs

In [36]:
songs = get_all_songs()
subset = songs[:200]

In [37]:
def find_bad_urls(songs):
  working = 0
  broken = 0
  for song in songs:
    # print(song['url'])
    r = requests.head(song['song_url'], allow_redirects=True) # the limitation here is that if the link is valid and the song is a normal length, it takes a long time for the request to complete
    if r.status_code == 403:
      broken = broken + 1
      song['corrupted'] = 'BadUrl'
    elif r.status_code == 200: 
      working = working + 1
    # print(r.status_code)
    print('{} working. {} broken'.format(working, broken), end='\r')
  return songs


In [38]:
stuff = find_bad_urls(subset)

In [39]:
stuff_df = pd.DataFrame(stuff)

In [40]:
from sqlalchemy import create_engine
engine = create_engine("mysql+pymysql://{}:{}@deadhead-db.cplvgriavgfs.us-east-1.rds.amazonaws.com:3306/deadhead".format(db_username, db_password))

In [41]:
stuff_df.to_sql('song',engine, if_exists='replace', index_label='song_id')

200

In [ ]:
conn = get_connection()
with conn:
  with conn.cursor() as cursor:
      query = "SELECT * FROM song WHERE `corrupted` = 'NO' ORDER BY RAND() LIMIT 1"
      cursor.execute(query)
      song = cursor.fetchone()
      print(song)
      
      query = "SELECT * FROM concert WHERE `concert_id` = {} ".format(song['concert_id'])
      cursor.execute(query)
      concert = cursor.fetchone()
      print(concert)
      print({**song, **concert})

{'id': 1610, 'title': '11. Fire On The Mountain', 'format': 'VBR MP3', 'archive_id': '/11. Fire On The Mountain.mp3', 'url': 'https://ia804601.us.archive.org/0/items/gd1988-12-09.151088.aud.lohr.vernon.flac1648/11. Fire On The Mountain.mp3', 'concert_id': 61, 'corrupted': 'No'}
{'concert_id': 61, 'title': 'Grateful Dead Live at Long Beach Arena on 1988-12-09', 'year': 1988, 'coverage': 'Long Beach, CA', 'identifier': 'gd1988-12-09.151088.aud.lohr.vernon.flac1648', 'date': '1988-12-09T00:00:00Z'}
{'id': 1610, 'title': 'Grateful Dead Live at Long Beach Arena on 1988-12-09', 'format': 'VBR MP3', 'archive_id': '/11. Fire On The Mountain.mp3', 'url': 'https://ia804601.us.archive.org/0/items/gd1988-12-09.151088.aud.lohr.vernon.flac1648/11. Fire On The Mountain.mp3', 'concert_id': 61, 'corrupted': 'No', 'year': 1988, 'coverage': 'Long Beach, CA', 'identifier': 'gd1988-12-09.151088.aud.lohr.vernon.flac1648', 'date': '1988-12-09T00:00:00Z'}
